In [9]:
import yaml
import os
from pathlib import Path
import pandas as pd

## Setup

In [3]:
pdk = 'sky130'
design = 'add'
design_ext = '8'
inst = 'adder0'
obj_dir = f'build-{pdk}-cm/add8'
e2e_dpath = Path(os.getcwd()).parent
obj_dpath = e2e_dpath/obj_dir
tests_dpath = e2e_dpath/'experiments/tests'

## Generate experiment files

What do we want to test?

Inputs:
* 0 -> 0
* 0 -> 11...11
* different activity factors of adds

Designs:
* minimum critical path - can we force the synthesis tool to use faster gates

Flow:
* sim-rtl   > 
* synthesis > sim-gl-syn  > power-syn

In [4]:
from pathlib import Path
testnames = ['zero','max_input_switching','max_output_switching']
my_tests = [f"{design}{design_ext}-{t}" for t in testnames]
tests_dict = {t: {} for t in my_tests}
# create dirs
for t in tests_dict:
    tests_dict[t]['dpath'] = tests_dpath/t
    tests_dict[t]['dpath'].mkdir(exist_ok=True)

# set operands
tests_dict['add8-zero']['inputs'] = [(0,0) for _ in range(50)]
tests_dict['add8-max_input_switching']['inputs'] = [((0,0) if i%2 else (255,255)) for i in range(50)]
tests_dict['add8-max_output_switching']['inputs'] = [((0,0) if i%2 else (127,128)) for i in range(50)]

# write out input.txt
for t in tests_dict:
    with (tests_dict[t]['dpath']/'input.txt').open('w') as f:
        for i in tests_dict[t]['inputs']:
            f.write(" ".join(['{0:b}'.format(ii) for ii in i]) + '\n')


In [5]:
def write_cfg(test_dict):
    cfg = f"""\
design.defines: &DEFINES
  - CLOCK_PERIOD=10
  - WIDTH=8
  - IN0_R=0
  - IN1_R=0
  - TESTROOT={test_dict['dpath']}

sim.inputs:
  defines: *DEFINES
  defines_meta: 'append'

synthesis.inputs:
  defines: *DEFINES

power.inputs:
  defines: *DEFINES

vlsi.core.power_tool: hammer.power.joules
power.inputs:
  level: rtl
  input_files: [src/add.v]
  report_configs:
    - waveform_path: {test_dict['dpath']}/output.fsdb
      report_stem: {test_dict['dpath']}/power
      toggle_signal: clock
      num_toggles: 1
      levels: all
      output_formats:
      - report
      - plot_profile
"""
    with (test_dict['dpath']/'config.yml').open('w') as f:
        f.write(cfg)

for t in tests_dict:
    write_cfg(tests_dict[t])

## Run experiments

In [6]:
# generate custom make str for each test
for t in tests_dict:
    cfg = str(tests_dict[t]['dpath']/'config.yml')
    tests_dict[t]['make'] = f"make design={design} OBJ_DIR={obj_dir} extra={cfg}"
# build
print(f"make design={design} OBJ_DIR={obj_dir} build -B\n")

# sim-rtl
for t in tests_dict:
    print(f"{tests_dict[t]['make']} redo-sim-rtl")
print()

# power-rtl
for t in tests_dict:
    make_target = "redo-power-rtl args='--only_step report_power'" \
            if (obj_dpath/'power-rtl-rundir/pre_report_power').exists() else 'power-rtl'
    print(f"{tests_dict[t]['make']} {make_target}")
print()

make design=add OBJ_DIR=build-sky130-cm/add8 build -B

make design=add OBJ_DIR=build-sky130-cm/add8 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests/add8-zero/config.yml redo-sim-rtl
make design=add OBJ_DIR=build-sky130-cm/add8 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests/add8-max_input_switching/config.yml redo-sim-rtl
make design=add OBJ_DIR=build-sky130-cm/add8 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests/add8-max_output_switching/config.yml redo-sim-rtl

make design=add OBJ_DIR=build-sky130-cm/add8 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests/add8-zero/config.yml redo-power-rtl args='--only_step report_power'
make design=add OBJ_DIR=build-sky130-cm/add8 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests/add8-max_input_switching/config.yml redo-power-rtl args='--only_step report_power'
make design=add OBJ_DIR=build-sky130-cm/add8 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests/add8-max_out

## Parse results

In [12]:

def parse_hier_power_rpt(fpath,inst) -> list:
    with fpath.open('r') as f: lines = f.readlines()
    for l in lines:
        words = l.split()
        if inst == words[-1]:
            return [round(float(p),5) for p in words[2:6]]

power = []
for t in tests_dict:
    fpath = tests_dict[t]['dpath']/'power.hier.power.rpt'
    power.append(parse_hier_power_rpt(fpath,f"/{design}/{inst}"))

power = pd.DataFrame(power,
                     columns=['Leakage','Internal','Switching','Total'],
                     index=tests_dict.keys())
power

,Leakage,Internal,Switching,Total
add8-zero,0.0,0.00000,0.00000,0.00000
add8-max_input_switching,0.0,0.06106,1.13899,1.20005
add8-max_output_switching,0.0,0.05402,1.28806,1.34208
